<h1 align='center'>Laboratorio de Fuga</h1>

<h2>1. Importación de Librerías</h2>

Para simplificar el ejercicio, primero importaremos las librerías, sin necesidad de levantar un ambiente virtual e instalarlas en éste, gracias a las virtudes de Google Colaboratory como editor. Otra de las ventajas de  este intérprete de Python, es que funciona con el formato de celdas de los *Jupyter Notebooks*. Esto permite trabajar bajo el paradigma de lo que se denomina *Literate Programming*, pudiéndose hacer un claro énfasis en la estructura lógica del programa.

Para ejecutar la celda a continuación, bastará que usted la seleccione apretando sobre ella con el cursor, y luego apriete <code>shift+enter</code>




In [161]:
# Statistical Libraries
import numpy as np
import statsmodels.api as sm

# Operational Libraries
import pandas as pd
from typing import Optional
import datetime

<h2>2. Lectura de la Tabla de Datos</h2>

Para ejercitar, utilizaremos una tabla de datos (en adelante <i>dataset</i>) proporcionada por la empresa de Telecom, perteneciente a la industria de las telecomunicaciones en Chile. Los datos no han sido procesados, por lo que hay ciertos campos que se transformarán y otros tantos que enriqueceremos en el proceso de segmentación.

Es importante notar que hay una columna numérica de identificación de los sujetos, denominada "Identificador". No se confunda, esta variable es arbitraria para todos los efectos prácticos, y no debe ser considerada para el análisis, salvo que sea considerada como llave relacional. Si usted no se encuentra familiarizado con el lenguaje y las librerías, Pandas incluye siempre un índice que comúnmente es también numérico. No confunda el índice con la columna de identificación.

In [162]:
#Importing the prospects dataset using pandas
file_path = f'propuesto_tymo.xlsx'
dataset_tymo = pd.read_excel(file_path, sheet_name='propuesto_tymo')

# Display of last 5 rows
dataset_tymo.head(5)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0


Vamos a analizar las distintas columnas de las que disponemos, utilizando funciones como las que se muestran a continuación:

In [163]:
for col in dataset_tymo.columns:
    unique_values = dataset_tymo[col].unique()

    if len(unique_values) < 5:
        print(f'{col}: {unique_values}')

    else:
        print(f'{col}: {unique_values[:5]} (muestra)')

CustomerId: [15634602 15647311 15619304 15701354 15737888] (muestra)
Surname: ['Hargrave' 'Hill' 'Onio' 'Boni' 'Mitchell'] (muestra)
CreditScore: [619 608 502 699 850] (muestra)
Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']
Age: [42 41 39 43 44] (muestra)
Tenure: [2 1 8 7 4] (muestra)
Balance: [     0.    83807.86 159660.8  125510.82 113755.78] (muestra)
NumOfProducts: [1 3 2 4]
HasCrCard: [1 0]
IsActiveMember: [1 0]
EstimatedSalary: ['101348.88' '112542.58' '113931.57' '93826.63' '79084.1'] (muestra)
Exited: [1 0]


Gracias al análisis anterior, podemos tener una visión más clara respecto al contenido de cada columna, pudiéndose construir una tabla que las describra, como se muestra a continuación (Pendiente: crear nueva tabla):

| Columna | Nombre de la Variable | Contenido de la Columna | Medida o Alternativas |
|---------|----------------------|-------------------------|-----------------------|
| 1       | customerID           | Identificador único del cliente | 15634602, 15647311, 15619304, 15701354, ... |
| 2       | Surname              | Apellido del cliente | Hargrave, Hill, Onio, Boni, ... |
| 3       | CreditScore          | Puntaje de crédito del cliente | 619, 608, 502, 699, 850 ... |
| 4       | Geography            | País de residencia del cliente | France, Spain, Germany |
| 5       | gender               | Género del cliente | Female, Male |
| 6       | Age                  | Edad del cliente | 42, 41, 39, 43, 44, ... |
| 7       | Tenure               | Número de meses que el cliente ha estado en el servicio | 2, 1, 8, 7, 4, ... |
| 8       | Balance              | Saldo de la cuenta del cliente | 0.00, 83807.86, 159660.80, 125510.82, 113755.78, ... |
| 9       | NumOfProducts        | Número de productos que el cliente tiene contratados | 1, 3, 2, 4 |
| 10      | HasCrCard            | Indica si el cliente tiene tarjeta de crédito | 1, 0 |
| 11      | IsActiveMember       | Indica si el cliente es un miembro activo | 1, 0 |
| 12      | EstimatedSalary      | Salario estimado del cliente | 101348.88, 112542.58, 113931.57, 93826.63, 79084.1, ... |
| 13      | Exited               | Indica si el cliente ha abandonado el servicio | 1, 0 |



<h2>3. Transformación de los Datos</h2>

A continuación procederemos a transformar los datos, creando un dataset numérico a partir del que descargamos. Si bien es más costoso en memoria el uso de réplicas completas de los datasets utilizados, esta práctica es conveniente cuando se trabaja con Jupyter Notebooks que podrían ser ejecutados en desorden o múltiples veces.

In [164]:
# Copying our dataset to avoid future issues
dataset_tymo_numerico = dataset_tymo.copy(deep=True)

Recordemos que los datos recolectados en una encuesta pueden ser de cuatro tipos principalmente:

- Nominal: nombres (identificación y clasificación)
- Ordinal: orden (jerarquización, posición relativa)
- Intervalo: cuantificación (cero arbitrario)
- Escala: cuantificación (cero absoluto)

Es importante que usted identifique el tipo de cada variable, para que así le sea más sencillo transformar los datos en información valiosa para su posterior análisis. Para efectos de este ejemplo, dividiremos la transformación de los datos por tipo para mayor claridad, transformando aquellas variables de tipo texto en numéricas según corresponda.

In [165]:
cols_nominal = [ 'CustomerId', 'Surname', 'Geography', 'Gender', 'HasCrCard', 'IsActiveMember', 'Exited']

cols_escala = [ 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

# We didn't find any ordinal columns in the dataset we are working with
# We also didn't find any interval columns in the dataset we are working with

<h3>3.1. Valores Nominales</h3>

En primera instancia, se ha decidido que la columna "Surname" no contiene información relevante para el análisis de fuga de clientes, por lo que se eliminará del dataset. Ya contamos con la variable "customerID" como identificador exclusivo de los clientes, por lo que no necesitamos su apellido.

In [166]:
dataset_tymo_numerico.drop(columns=['Surname'], inplace=True)
cols_nominal.remove('Surname')
dataset_tymo_numerico.head(5)

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0



Convenientemente, Pandas puede transformar cualquier variable de tipo nominal en una variable categórica por nosotros. Sin embargo, este proceso no está excento de errores y debe ser monitoreado de cerca. Para controlar la calidad de los datos, almacenaremos las categorías en una lista con un diccionario que nos permitirá mapear cada uno de los valores.

In [167]:
print(f'Identificamos {len(cols_nominal)} variables nominales de forma manual\n'
      'A continuación procedereos a transformarlas a variables numéricas\n')
dict_mappers = dict()
counter = 0

for col in cols_nominal:
    # Create and store our mapper dictionary, and print it for analysis
    mapping = dict(enumerate(dataset_tymo_numerico[col].astype('category').cat.categories))
    dict_mappers.update({col: {value: key for key, value in mapping.items()}})
    counter += 1

    # Print the mapper dictionary for max 5 entries of the dictionary
    mapper_print = {entrance for i, entrance
                    in enumerate(dict_mappers[col]) if i < 6}

    print(f'{counter}) {col}: {mapper_print}')

Identificamos 6 variables nominales de forma manual
A continuación procedereos a transformarlas a variables numéricas

1) CustomerId: {15565796, 15565701, 15565706, 15565806, 15565714, 15565779}
2) Geography: {'France', 'Spain', 'Germany'}
3) Gender: {'Male', 'Female'}
4) HasCrCard: {0, 1}
5) IsActiveMember: {0, 1}
6) Exited: {0, 1}


Podemos apreciar que nos quedan dos variables nominales en formato de texto: "Geography" y "Gender". Por lo tanto, vamos a convertirlas en categorías numéricas de forma manual.

In [168]:
geography_mapping = { country: key for key, country in enumerate(dataset_tymo_numerico['Geography'].astype('category').cat.categories)}
dataset_tymo_numerico['Geography'] = dataset_tymo_numerico['Geography'].map(geography_mapping)
dataset_tymo_numerico.head(5)

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,619,0,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,2,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,0,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,0,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,2,Female,43,2,125510.82,1,1,1,79084.1,0


In [169]:
gender_mapping = { gender: key for key, gender in enumerate(dataset_tymo_numerico['Gender'].astype('category').cat.categories)}
dataset_tymo_numerico['Gender'] = dataset_tymo_numerico['Gender'].map(gender_mapping)
dataset_tymo_numerico.head(5)

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,2,0,43,2,125510.82,1,1,1,79084.1,0


Ahora podemos reemplazar los valores de cada columna por su equivalente numérico pasando por el mapper adecuado.

In [170]:
for col, mapper in dict_mappers.items():
    dataset_tymo_numerico[col] = dataset_tymo[col].map(mapper)

Antes de seguir con las siguientes variables, vamos a verificar que el tipo de variables sea numérica.

In [171]:
dataset_tymo_corregido = dataset_tymo_numerico.copy(deep=True)
print(dataset_tymo_corregido.dtypes)

CustomerId           int64
CreditScore          int64
Geography            int64
Gender               int64
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary     object
Exited               int64
dtype: object


Observamos que la columna "EstimatedSalary" es de tipo object, por lo que vamos a transformarla a tipo float. 
Primero vamos a verificar si hay valores de tipo datetime en la columna, y sus índices.

In [172]:
series_tymo = dataset_tymo_corregido['EstimatedSalary']

# Check the data type of each value in the Series
type_series = series_tymo.apply(type)

# Filter the Series to keep only datetime values
datetime_values = series_tymo[type_series == datetime.datetime]

# Print the problematic values and their indices
print(datetime_values)

126     8636-05-01 00:00:00
172     5472-07-01 00:00:00
200     5978-02-01 00:00:00
551     2079-02-01 00:00:00
627     7698-06-01 00:00:00
               ...         
9568    9984-04-01 00:00:00
9679    3453-04-01 00:00:00
9745    8037-03-01 00:00:00
9754    3258-06-01 00:00:00
9811    2850-01-01 00:00:00
Name: EstimatedSalary, Length: 80, dtype: object


Notamos que contamos con 85 filas donde el valor de EstimatedSalary está contaminado con valores de formato datetime.datetime. Vamos a limpiar estos valores más adelante.

<h3>3.2. Variables Ordinales</h3>

No identificamos variables ordinales para este ejercicio, aunque se podría argumentar que el tipo de contrato se podría ordenar de menor a mayor duración, cuestión que en nuestro caso pasó por defecto. En tal caso, la única diferencia es que tendríamos que aplicar los mappers en forma manual, forzando así nuestro criterio subjetivo sobre los datos.

<h3>3.3. Variables de Intervalo</h3>

Comúnmente las encuestas contendrán preguntas de tipo Likert con alternativas que buscan evaluar la opinión o satisfacción de los clientes respecto a ciertos tópicos. En este caso, no se dispone de este tipo de preguntas, cuestión que de cara a determinar la fuga de clientes podría ser perjudicial, pues contar con variables psicográficas o conductuales de los clientes no es sólo importante para la segmentación, sino también para este tipo de ejercicios.

<h3>3.4. Variables de Escala</h3>

También de libro, las variables de escala suelen ser la edad y montos finitos como en este caso el costo de los planes. Convenientemente, como todas estas variables ya son numéricas y contínuas, no debiéramos realizar ningún tipo de transformación. Sin embargo, como observaremos a continuación, las columnas asociadas son de tipo "object", cuestión que hace referencia a que los valores se almacenaron como texto y no como número.

In [173]:
mapper_escala = {'EstimatedSalary': float}

for col, dtype in mapper_escala.items():
    if dtype == float:
        dataset_tymo_numerico[col] = dataset_tymo_numerico[col].str.replace(',', '.').astype(dtype)
    else:
        dataset_tymo_numerico[col] = dataset_tymo_numerico[col].astype(dtype)

    print(f"{col}: {dataset_tymo_numerico[col].dtype}")

EstimatedSalary: float64


In [174]:
dataset_tymo_numerico.iloc[126]

CustomerId         4209.0
CreditScore         549.0
Geography             0.0
Gender                0.0
Age                  52.0
Tenure                1.0
Balance               0.0
NumOfProducts         1.0
HasCrCard             0.0
IsActiveMember        1.0
EstimatedSalary       NaN
Exited                1.0
Name: 126, dtype: float64

Verificamos qué ocurre con los valores que anteriormente presentaban un formato de fecha en la columna "EstimatedSalary".
Podemos apreciar que ahora los valores de la columna "EstimatedSalary" son de tipo float. Sin embargo, los datos que antes tenían formato datetime, ahora son valores nulos. Esto lo vamos a seguir preparando para el análisis en los siguientes pasos.

<h2>4. Ejecución de las Regresiones Logísticas</h2>

La regresión logística es un modelo estadístico vital para predecir la probabilidad de fuga de clientes, un fenómeno también conocido como churn, que afecta significativamente los ingresos y la rentabilidad de las empresas. Este modelo predice una variable categórica binaria a partir de variables independientes que describen características y comportamientos de los clientes, permitiendo calcular la probabilidad de fuga. Al aplicar regresiones logísticas, es posible identificar las variables con mayor impacto en la fuga, lo que ayuda a las empresas a comprender y mitigar los factores que la propician. Además, este modelo facilita la generación de una puntuación de probabilidad de fuga por cliente, optimizando las estrategias de retención al priorizar y personalizar las intervenciones. En definitiva, la regresión logística ofrece una herramienta esencial para la toma de decisiones informadas y la implementación de acciones efectivas para retener a los clientes en riesgo.

<h3>4.1. Prueba inicial y Correcciones</h3>

La idea de aquí en adelante es ir reduciendo las variables significativas para el análisis, mediante un proceso de ensayo y error en el que se van eliminando aquellas variables que presenten p-values no significativos. Sin embargo, siempre es bueno probar inicialmente a ver si nuestras columnas se acomodan al modelo estadístico.

In [175]:
dataset_tymo_regression = dataset_tymo_numerico.copy(deep=True)

try:
    # Run a logistic regression model with all variables and the churn as the target
    x = dataset_tymo_regression.drop(columns=['CustomerId', 'Exited'])
    y = dataset_tymo_regression['Exited']

    x = sm.add_constant(x)
    model = sm.Logit(y, x)
    result = model.fit()

    print(result.summary())

except Exception as e:
    print(e)

exog contains inf or nans


En este caso, observamos que las variables exógenas presentan valores infinitos o nulos. Revisaremos la estadísticas descriptiva de la tabla para revisar los máximos y descartar al primer sospechoso de que no podamos correr el modelo.

In [176]:
dataset_tymo_regression.describe()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,9801.000000,10000.000000
mean,4999.50000,650.528800,0.746300,0.545700,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,101065.106289,0.203700
std,2886.89568,96.653299,0.827529,0.497932,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57100.263571,0.402769
min,0.00000,350.000000,0.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2499.75000,584.000000,0.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,52449.620000,0.000000
50%,4999.50000,652.000000,0.000000,1.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,101139.300000,0.000000
75%,7499.25000,718.000000,1.000000,1.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149705.250000,0.000000
max,9999.00000,850.000000,2.000000,1.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Dado que no hay valores infinitos, seguramente tengamos valores nulos en alguna columna. Este análisis es un poco más complejo, pero nada que no se pueda resolver consultando a Chat GPT si no supiéramos cómo hacerlo. A continuación, revisamos cada columna y buscamos la presencia de valores nulos.

In [177]:
for col in dataset_tymo_regression.columns:
    if dataset_tymo_regression[col].isna().sum() > 0:
        print(f'{col}: {dataset_tymo_regression[col].isna().sum()}')



EstimatedSalary: 199


In [178]:
display(dataset_tymo_regression[dataset_tymo_regression['EstimatedSalary'].isna()][['EstimatedSalary', 'Tenure', 'Balance', 'NumOfProducts']])

,EstimatedSalary,Tenure,Balance,NumOfProducts
69,NaN,8,98373.26,1
75,NaN,1,178718.19,2
126,NaN,1,0.00,1
172,NaN,10,129608.57,1
200,NaN,2,141040.01,1
...,...,...,...,...
9684,NaN,3,89276.48,1
9745,NaN,4,0.00,2
9754,NaN,1,121629.22,1
9811,NaN,1,128927.93,1


Al parecer, la presencia de valores NaN (anteriormente valores de tipo datetime) en la columna "EstimatedSalary" no tiene alguna correlación con las demás variables del dataset. Como no hay una forma clara de extraer implicancias en los datos, decidimos eliminar las filas con valores nulos.

In [179]:
dataset_tymo_regression_dropna = dataset_tymo_regression.dropna()

In [180]:
dataset_tymo_regression_dropna.describe()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,9801.000000,9801.000000,9801.000000,9801.000000,9801.000000,9801.000000,9801.000000,9801.000000,9801.000000,9801.000000,9801.000000,9801.000000
mean,5001.695949,650.622998,0.747577,0.546067,38.944393,5.022141,76605.263232,1.530456,0.707071,0.515356,101065.106289,0.203449
std,2885.551049,96.507401,0.827925,0.497899,10.494822,2.889322,62425.080026,0.581971,0.455129,0.499790,57100.263571,0.402584
min,0.000000,350.000000,0.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,2505.000000,584.000000,0.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,52449.620000,0.000000
50%,4997.000000,652.000000,0.000000,1.000000,37.000000,5.000000,97378.540000,1.000000,1.000000,1.000000,101139.300000,0.000000
75%,7499.000000,718.000000,1.000000,1.000000,44.000000,8.000000,127758.090000,2.000000,1.000000,1.000000,149705.250000,0.000000
max,9999.000000,850.000000,2.000000,1.000000,92.000000,10.000000,238387.560000,4.000000,1.000000,1.000000,199992.480000,1.000000


Hint: "puede" que en el laboratorio que usted deba resolver no sean ni los valores infinitos ni los valores nulos el problema. "Quizás" haya variables fuertemente correlacionadas que "probablemente" usted podría identificar corriendo un correlograma. Se "sugiere" ese método para botar las variables problemáticas, aunque encontraría genial si usted identifica la verdadera causa del problema.

<h3>4.2. Ejecución de la regresión principal</h3>

Para las siguientes líneas, por favor asegúrese de instalar matplotlib y jinja2

In [185]:
dataset_tymo_optimized = dataset_tymo_regression_dropna.copy(deep=True)
dataset_tymo_optimized.drop(columns=['CustomerId'], inplace=True)
confidence_level = 0.05
counter = 1

def optimize_regression(dataset, confidence_level: Optional[float] = 0.05):
    # Drop the constant column
    if 'const' in dataset.columns: # if it exists
        dataset.drop(columns=['const'], inplace=True)

    # Run a logistic regression model with all variables and the churn as the target
    x = dataset.drop(columns=['Exited'])
    y = dataset['Exited']

    x = sm.add_constant(x)
    model = sm.Logit(y, x)
    result = model.fit(disp=False)
    drop_cols = {}

    # Access the p-values of the model
    for index in result.pvalues.index:
        if result.pvalues[index] >= confidence_level:
            drop_cols[index] = result.pvalues[index]

    return (result, {col: p_value for col, p_value in drop_cols.items() if col != 'const'})

while True:
    print(f'Iteración {counter}:\n')
    result, drop_cols = optimize_regression(dataset_tymo_optimized, confidence_level)
    print(result.summary())

    if len(drop_cols) == 0:
        print('\nEl modelo ha sido exitosamente optimizado\n')
        break
    else:
        counter += 1

    # Drop the column with the highest p-value
    column_to_drop = max(drop_cols, key=drop_cols.get)
    dataset_tymo_optimized.drop(columns=column_to_drop, inplace=True)
    drop_cols.pop(column_to_drop)

    print(f'\nSe eliminó la columna {column_to_drop}\n\n')

Iteración 1:

                           Logit Regression Results                           
Dep. Variable:                 Exited   No. Observations:                 9801
Model:                          Logit   Df Residuals:                     9790
Method:                           MLE   Df Model:                           10
Date:                Mon, 27 May 2024   Pseudo R-squ.:                  0.1388
Time:                        01:20:47   Log-Likelihood:                -4263.9
converged:                       True   LL-Null:                       -4950.9
Covariance Type:            nonrobust   LLR p-value:                3.825e-289
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -3.5376      0.245    -14.440      0.000      -4.018      -3.057
CreditScore        -0.0006      0.000     -2.255      0.024      -0.001   -8.29e-05
Geography         

<h2>Evaluar la efectividad del Modelo</h2>



Estas líneas de código evalúan nuestro modelo de predicción de fuga. La funcion evaluate_model convierte probabilidades en valores binarios, calcula la matriz de confusión. La matriz de confusión es clave, ya que muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos, permitiendo identificar donde el modelo acierta y falla, proporcionando información valiosa para evaluar financieramente el costo asociado a la fuga.

In [186]:
# Definir la función de evaluación
def evaluate_model(model, x, y):
    # Obtener los valores predichos
    y_pred_prob = model.predict(sm.add_constant(x))

    # Convertir las probabilidades a valores binarios
    y_pred = [1 if value > 0.5 else 0 for value in y_pred_prob]

    # Calcular la matriz de confusión
    confusion_matrix = pd.crosstab(y, y_pred, rownames=['Actual'], colnames=['Predicted'])

    # Calcular la exactitud
    accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1]) / len(y)

    return accuracy, confusion_matrix

# Preparar los datos (usando el mismo conjunto de datos que el modelo entrenado)
x = dataset_tymo_optimized.drop(columns=['Exited'])
y = dataset_tymo_optimized['Exited']

# Evaluar el modelo
accuracy, confusion_matrix = evaluate_model(result, x, y)
print(f'La exactitud del modelo es de {accuracy * 100:.2f}%\n')
display(confusion_matrix)

La exactitud del modelo es de 80.76%



Predicted,0,1
Actual,,
0,7562,245
1,1641,353


<h2>Efectos Porcentuales</h2>

Por último, transformaremos los coeficientes en odds ratios y en porcentajes, para poder interpretar de mejor manera el efecto de cada factor sobre la fuga de clientes.

In [187]:
# Obtener los coeficientes
coefficients = result.params

# Calcular los odds ratios
odds_ratios = np.exp(coefficients)

# Calcular los porcentajes de cambio
percentages = (odds_ratios - 1) * 100

# Mostrar los resultados
results_df = pd.DataFrame({
    'Coefficient': coefficients,
    'Odds Ratio': odds_ratios,
    'Percentage Change': percentages
})

print(results_df)

                Coefficient  Odds Ratio  Percentage Change
const             -3.642751    0.026180         -97.381977
CreditScore       -0.000640    0.999360          -0.063966
Geography          0.078973    1.082175           8.217493
Gender            -0.536416    0.584841         -41.515916
Age                0.072620    1.075322           7.532192
Balance            0.000005    1.000005           0.000508
IsActiveMember    -1.076373    0.340830         -65.917037


<h2>Respuesta a la pregunta final del ejercicio</h2>

¿Cómo podríamos utilizar un modelo de regresión logística para desarrollar una herramienta de predicción continua que alerte sobre clientes potencialmente fugitivos?
Integrar el modelo en el sistema CRM permite monitorear las probabilidades de fuga en tiempo real y configurar alertas automatizadas. Esto facilita la segmentación de clientes y la personalización de estrategias de retención basadas en factores específicos que contribuyen a la probabilidad de fuga.